In [19]:
import numpy as np
import pandas as pd
import networkx as nx
import traceback
import os
import matplotlib.pyplot as plt
import math
from ant_defs import Ant
import time
import tsplib95
from ant_defs import observer
from ant_defs import makestatsfile

In [20]:
def findbest(pop: list):
    best = Ant(0)
    best.setTourLength(100000000.0)
    for ant in pop:
        if ant.isTourViable():
            if ant < best:
                best = ant
    return best

==============================================================================

ANT SYSTEM

======================================================================================

In [21]:
#TODO; THIS SELECTION IS IN SUPER HELL
# TO REWRITW
def ASNextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    psum = 0
    pvals = []
    for node in viable_paths:
        tmp = math.pow(graph[visited[-1]][node]["heuristic"],args['heuristic_preference']) + math.pow(graph[visited[-1]][node]["pheromone"],args['pheromone_preference'])
        pvals.append(tmp)
    s = np.sum(pvals)
    probs = [x/s for x in pvals]
    #print(probs)
    next_node = np.random.choice(a=viable_paths,p=probs,replace=False)
    #next_node = viable_paths[np.argmax(probs)]
    #print(next_node)
    #return viable_paths[np.argmax(pvals)]
    return next_node

def ASGlobalUpdate(graph: nx.DiGraph,population: list, args: dict) -> nx.DiGraph:
    for ant in population:
        #proper solutions only
        if ant.isTourViable(): 
            tour = ant.getTour()
            startnodes = tour[:-1]
            endnodes = tour[1:]
            for s,n in zip(startnodes,endnodes):
                graph[s][n]["delta"] = graph[s][n]["delta"] + (args['Qval'] / ant.getTourLength())
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [22]:
def AntSystemTSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(1)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(1)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        print("ant invalidated")
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        localbest = findbest(population)
        #print(localbest.getTourLength())
        #print(consecutive_same_best)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ASGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'AS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ANT COLONY ACO

=====================================================================================

In [23]:
def ACONextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    pvals = []
    if args['selectionTypeConst'] > np.random.uniform():
        for node in viable_paths:
            tmp = graph[visited[-1]][node]["heuristic"] + math.pow(graph[visited[-1]][node]["pheromone"],args['heuristic_preference'])
            pvals.append(tmp)
        return viable_paths[np.argmax(pvals)]
    else:
        return ASNextNodeSelection(graph,viable_paths,visited,args)

def ACOGlobalPheromoneUpdate(graph: nx.DiGraph,population: list, args: dict):
    evap_coef = 1-args['evaporation']
    best = args['globalbest']
    tour = best.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += math.pow(best.getTourLength(),-1)
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = (evap_coef*graph[u][v]["pheromone"]) + (args['evaporation']*graph[u][v]["delta"])
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

def ACOLocalUpdateTau0(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    one = evap_coef*graph[lastnode][newnode]['pheromone']
    two = args['evaporation']*args['startpheromone']
    graph[lastnode][newnode]["pheromone"] = one + two
    return graph

def ACOLocalUpdateQlearning(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    nextnodes = [x for x in [n for n in graph[newnode]] if x not in taboo]
    if len(nextnodes) == 0:
        return ACOLocalUpdateTau0(graph,lastnode,newnode,taboo,args)
    else:
        nextphero = []
        for node in nextnodes:
            nextphero.append(graph[newnode][node]['pheromone'])
        delta = args['ACS_qlearn_gamma']*np.max(nextphero)
        one = evap_coef*graph[lastnode][newnode]['pheromone']
        two = args['evaporation']*delta
        graph[lastnode][newnode]["pheromone"] = one + two
        return graph

In [24]:
def ACO_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ACONextNodeSelection(problem,viable_paths,visited,argslocal)
                    ACOLocalUpdateQlearning(problem,visited[-1],next_node,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ACOGlobalPheromoneUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ACO', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

MMAS

=====================================================================================

In [25]:
def MMAStaumax(graph: nx.DiGraph, bestlength: float, args: dict):
    one = 1/(1-args['evaporation'])
    two = 1/bestlength
    max_pheromone = one * two
    args['max_pheromone'] = max_pheromone
    return args

def MMAStaumin(graph: nx.DiGraph, args: dict):
    n = graph.number_of_nodes()
    p = args['best_pheromone_selection_prob']**(1/n)
    av = n/2
    min_pheromone = (args['max_pheromone']*(1-p))/((av-1)*p)
    args['min_pheromone'] = min_pheromone
    return args

def MMASTrailSmoothing(graph: nx.DiGraph, args: dict):
    for u, v, pheromone in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = pheromone + args['smoothing_coef']*(args['max_pheromone']-pheromone)
    return graph

def MMASGlobalUpdate(graph: nx.DiGraph,localbest, args: dict) -> nx.DiGraph:
        #proper solutions only
    tour = localbest.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += args['Qval'] / localbest.getTourLength()
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        new_pheromone = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
        if new_pheromone > args['max_pheromone']:
            new_pheromone = args['max_pheromone']
        elif new_pheromone < args['min_pheromone']:
            new_pheromone = args['min_pheromone']
        graph[u][v]["pheromone"] = new_pheromone
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [26]:
def MMAS_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    max_stagnations = 2
    current_stagnations = 0
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        argslocal = MMAStaumax(problem,globalbest.getTourLength(),argslocal)
        argslocal = MMAStaumin(problem,argslocal)
        problem = MMASGlobalUpdate(problem,localbest,argslocal)
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        if consecutive_same_best > 10:
            stagnation = i
            problem = MMASTrailSmoothing(problem,argslocal)
            consecutive_same_best = 0
            current_stagnations = current_stagnations + 1
        if current_stagnations > max_stagnations:
            break
        # TODO: Find way to detect convergence and stagnation to insert trail smoothing here
    end = time.time()
    extime = end - start
    observer(stats_file,'MMAS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ASRANK

=====================================================================================

In [27]:
def ASRANKGlobalUpdate(graph,population,args):
    good_solutions = [x for x in population if x.isTourViable()]
    good_solutions.sort(reverse=True)
    ranks = args['elite_multiplier']-1
    used_solutions = good_solutions[-ranks:]

    for rank, ant in enumerate(used_solutions):
        r_rank = rank + 1
        tour = ant.getTour()
        startnodes = tour[:-1]
        endnodes = tour[1:]
        for s,n in zip(startnodes,endnodes):
            graph[s][n]["delta"] += (args['Qval'] / ant.getTourLength())*r_rank
    best = args['globalbest']   
    besttour = best.getTour()
    startnodes = besttour[:-1]
    endnodes = besttour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["elite_delta"] += (args['Qval'] / best.getTourLength())*args['elite_multiplier']
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"] + graph[u][v]["elite_delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    nx.set_edge_attributes(graph,0.0,"elite_delta")
    return graph

In [28]:
def ASRANK_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int, stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    nx.set_edge_attributes(problem,0.0,"elite_delta")
    population = []
    globalbest = Ant(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = Ant(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            population.append(Ant(1))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,problem[visited[-1]][next_node]["weight"])
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        #post cycle pheromone updates and solution assesment
        #review population
        for ant in population:
            visited = ant.getTour()
            ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 10:
            stagnation = i
            break
        problem = ASRANKGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ASRANK', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

In [29]:
stats_file = makestatsfile('bur26a.csv')

In [30]:
p = tsplib95.load('ft53.atsp')
p.edge_weights  
graph = p.get_graph()
nx.set_edge_attributes(graph,1.0,"pheromone")
nx.set_edge_attributes(graph,'b',"color")
nx.set_edge_attributes(graph,0.0,"heuristic")
nx.set_edge_attributes(graph,0.0,"delta")

for u, v, weight in graph.edges(data="weight"):
    if(weight is not None and weight != 0):
        print(f'source {u}, dest {v}, weight {weight}')
        graph[u][v]["heuristic"] = 1/weight

In [31]:
# from QAPInstances import getBur26a
# burA, burB = getBur26a()
# burA = np.matrix(burA)
# burB = np.matrix(burB)
# vectA = burA.sum(axis=1)
# vectB = burB.sum(axis=1)
# graphweights = np.dot(vectA,vectB.T)
# graph = nx.from_numpy_matrix(graphweights, create_using=nx.DiGraph)
# nx.set_edge_attributes(graph,1.0,"pheromone")
# nx.set_edge_attributes(graph,'b',"color")
# nx.set_edge_attributes(graph,0.0,"heuristic")
# nx.set_edge_attributes(graph,0.0,"delta")

# for u, v, weight in graph.edges(data="weight"):
#     if(weight is not None and weight != 0):
#         print(f'source {u}, dest {v}, weight {weight}')
#         graph[u][v]["heuristic"] = 1/weight

source 0, dest 0, weight 10606050
source 0, dest 1, weight 3794175
source 0, dest 2, weight 5366025
source 0, dest 3, weight 8076600
source 0, dest 4, weight 27004950
source 0, dest 5, weight 2327850
source 0, dest 6, weight 4969125
source 0, dest 7, weight 6811875
source 0, dest 8, weight 15239700
source 0, dest 9, weight 532350
source 0, dest 10, weight 2434950
source 0, dest 11, weight 6455925
source 0, dest 12, weight 3600450
source 0, dest 13, weight 11681775
source 0, dest 14, weight 4972275
source 0, dest 15, weight 1970325
source 0, dest 16, weight 28350
source 0, dest 17, weight 9913050
source 0, dest 18, weight 9753975
source 0, dest 19, weight 8380575
source 0, dest 20, weight 6728400
source 0, dest 21, weight 1855350
source 0, dest 22, weight 2549925
source 0, dest 23, weight 88200
source 0, dest 24, weight 141750
source 0, dest 25, weight 2230200
source 1, dest 0, weight 10606050
source 1, dest 1, weight 3794175
source 1, dest 2, weight 5366025
source 1, dest 3, weight 807

In [32]:
args_AS = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.5,
    'startpheromone': 2.0
}

args_ACO = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.5,
    'startpheromone': 2.0,
    'selectionTypeConst': 0.3,
    'ACS_qlearn_gamma': 0.5
}

args_MMAS = {
    'pheromone_preference': 10,
    'heuristic_preference': 6,
    'Qval': 1,
    'evaporation': 0.5,
    'startpheromone': 2.0,
    'best_pheromone_selection_prob': 0.05,
    'max_pheromone': 4.0,
    'min_pheromone': 0.5,
    'smoothing_coef': 0.5
}

args_ASRANK = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.5,
    'startpheromone': 1.0,
    'elite_multiplier': 25
}

# args_AS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0
# }

# args_ACO = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'selectionTypeConst': 0.5,
#     'ACS_qlearn_gamma': 0.5
# }

# args_MMAS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'best_pheromone_selection_prob': 0.05,
#     'max_pheromone': 2.0,
#     'min_pheromone': 0.5,
#     'smoothing_coef': 0.5
# }

# args_ASRANK = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'elite_multiplier': 62
# }
# kro124p - 36230
# ft53 - 6905
# ft70 - 1950
# ftv35 - 1473

In [33]:
for d in range(0,5):
    G, solution = AntSystemTSP(graph,26,1000,stats_file,args_AS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1]
100000000.0
[1]
100000000.0
[1]
100000000.0
[1]
100000000.0
[1]
100000000.0


In [16]:
for d in range(0,5):
    graph, solution = ACO_TSP(graph,53,1000,stats_file,args_ACO)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 8, 9, 6, 5, 51, 48, 49, 52, 50, 29, 28, 25, 27, 26, 3, 0, 36, 35, 40, 37, 18, 17, 16, 15, 39, 38, 45, 41, 43, 46, 44, 34, 32, 31, 33, 30, 13, 12, 14, 20, 21, 22, 42, 47, 7, 24, 23, 11, 10, 2, 4, 19, 1]
8684.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 33, 31, 26, 25, 27, 29, 28, 3, 0, 36, 35, 38, 45, 41, 47, 42, 43, 46, 44, 34, 32, 30, 11, 10, 14, 12, 13, 17, 16, 15, 40, 37, 18, 19, 9, 7, 21, 20, 39, 4, 2, 23, 22, 24, 1]
8604.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 29, 26, 25, 27, 28, 7, 9, 3, 0, 36, 35, 40, 37, 18, 17, 16, 15, 39, 4, 2, 11, 10, 12, 14, 13, 34, 32, 31, 33, 30, 38, 45, 41, 42, 46, 43, 44, 47, 23, 20, 21, 24, 22, 19, 1]
8301.0
[1, 8, 6, 5, 51, 48, 49, 50, 52, 33, 31, 26, 25, 27, 29, 28, 42, 41, 43, 45, 44, 34, 32, 30, 0, 36, 35, 40, 37, 18, 17, 16, 15, 9, 7, 21, 20, 39, 38, 4, 2, 3, 13, 11, 10, 14, 12, 24, 22, 47, 46, 23, 19, 1]
8629.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 33, 31, 26, 25, 27, 29, 28, 47, 46, 43, 41, 44, 34, 32, 30, 0, 2, 3, 13, 12, 14, 11, 10, 4, 17, 16, 15, 37, 35, 21,

In [17]:
for d in range(0,5):
    graph, solution = MMAS_TSP(graph,53,1000,stats_file,args_MMAS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 8, 9, 6, 5, 51, 48, 49, 52, 50, 33, 31, 30, 0, 41, 42, 46, 43, 45, 44, 34, 32, 26, 25, 27, 29, 28, 24, 22, 20, 21, 47, 7, 36, 35, 40, 37, 18, 17, 16, 15, 19, 11, 10, 12, 14, 13, 3, 2, 23, 39, 38, 4, 1]
8527.0
[1, 6, 5, 51, 48, 49, 52, 50, 26, 25, 27, 29, 28, 11, 10, 41, 43, 42, 46, 45, 44, 34, 32, 31, 33, 30, 0, 3, 2, 17, 16, 15, 37, 35, 40, 36, 21, 20, 39, 38, 4, 22, 19, 18, 8, 9, 7, 12, 14, 13, 23, 24, 47, 1]
8560.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 28, 25, 26, 27, 29, 34, 32, 33, 31, 30, 0, 36, 35, 37, 40, 38, 45, 41, 42, 46, 43, 47, 44, 23, 20, 11, 3, 2, 17, 16, 15, 18, 19, 7, 9, 13, 12, 14, 10, 39, 21, 24, 22, 4, 1]
8745.0
[1, 8, 5, 51, 48, 49, 52, 50, 29, 28, 25, 27, 26, 41, 42, 46, 43, 45, 47, 44, 34, 32, 31, 33, 30, 0, 3, 2, 17, 16, 15, 37, 35, 40, 36, 10, 12, 14, 13, 11, 6, 9, 7, 21, 20, 39, 38, 4, 22, 19, 18, 23, 24, 1]
8279.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 29, 28, 25, 27, 26, 34, 32, 31, 33, 30, 0, 3, 2, 15, 35, 40, 37, 21, 20, 39, 36, 38, 45, 41, 43, 42, 46, 44, 47, 7,

In [18]:
for d in range(0,5):
    graph, solution = ASRANK_TSP(graph,53,1000,stats_file,args_ASRANK)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[1, 8, 6, 5, 51, 48, 49, 52, 50, 33, 31, 30, 3, 0, 2, 15, 37, 35, 40, 38, 36, 32, 26, 25, 27, 29, 28, 10, 44, 41, 43, 42, 46, 45, 47, 34, 12, 14, 13, 11, 17, 16, 23, 20, 39, 4, 9, 7, 21, 24, 22, 19, 18, 1]
9105.0
[1, 8, 6, 5, 49, 48, 52, 50, 51, 45, 41, 43, 42, 46, 44, 34, 32, 26, 25, 27, 29, 28, 33, 31, 30, 0, 36, 35, 40, 37, 38, 10, 14, 20, 21, 9, 7, 2, 16, 15, 19, 18, 17, 11, 3, 13, 12, 24, 22, 47, 23, 39, 4, 1]
8989.0
[1, 0, 3, 2, 17, 16, 15, 52, 48, 49, 51, 50, 33, 31, 26, 25, 27, 29, 28, 7, 5, 9, 8, 6, 43, 41, 47, 44, 46, 45, 42, 34, 32, 30, 36, 35, 38, 37, 39, 40, 21, 20, 11, 12, 14, 13, 10, 4, 22, 19, 18, 23, 24, 1]
8949.0
[1, 8, 6, 5, 51, 48, 49, 50, 52, 33, 31, 26, 25, 27, 29, 28, 20, 21, 38, 36, 35, 37, 18, 17, 16, 15, 7, 9, 3, 0, 11, 10, 12, 14, 13, 34, 32, 30, 2, 41, 42, 43, 45, 44, 46, 47, 24, 22, 19, 23, 39, 40, 4, 1]
9101.0
[1, 8, 6, 5, 51, 48, 49, 52, 50, 32, 31, 33, 26, 25, 27, 29, 28, 21, 20, 11, 10, 12, 14, 13, 17, 15, 37, 35, 38, 36, 40, 39, 45, 41, 47, 42, 46, 43,